In [1]:
import os, psutil, gc
import time 
import json
import pprint

from collections import defaultdict
import random

In [2]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer
from vllm import LLM, SamplingParams, PoolingParams

from sal.config import Config
from sal.models.reward_models import PRM
from sal.utils.score import aggregate_scores

# from core.reward_models import RLHFFlow, SkyworkO1
from core import reward_models

from core import best_of_n
from utils.load_data import load_data_prm800k

In [3]:
import importlib

In [4]:
# base_dir
base_dir = '/groups/kjun/tnn/datasets/'

# dataset path
data_dir = base_dir + "/prm800k/math_splits"

# llm and prm path
llm_dir = base_dir + "/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.Q4_K_M.gguf"
prm_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data-GGUF/Llama3.1-8B-PRM-Deepseek-Data.Q4_K_M.gguf"

llm_tokenizer_dir = base_dir + "/Llama-3.2-1B-Instruct"
prm_tokenizer_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data"

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
if torch.cuda.is_available():
    GPUs = os.environ.get('CUDA_VISIBLE_DEVICES', "0").split(',')
    print(f"GPUs = {GPUs}")
else:
    print("CUDA is not available.") 

GPUs = ['0', '1', '2', '3']


In [6]:
importlib.reload(reward_models)

<module 'core.reward_models' from '/home/u20/tnguyen9210/tnn1/LLMs/llm-reasoning-methods/core/reward_models.py'>

In [7]:
prm_dir = base_dir + "/Skywork-o1-Open-PRM-Qwen-2.5-1.5B"

In [8]:
prm = reward_models.SkyworkO1(model_path=prm_dir, device_map="auto")
gc.collect();torch.cuda.empty_cache();
print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(2)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(3)/(1024**3))

/home/u20/tnguyen9210/tnn1/LLMs/search-and-learn/src/sal/models/skywork_o1_prm/modeling_base.py:309: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = loading_func(

#--- memory: 0.7842240333557129
#--- memory: 0.8716773986816406
#--- memory: 0.8716773986816406
#--- memory: 0.34867429733276367


In [9]:
# del(prm)
gc.collect();torch.cuda.empty_cache();
print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))

#--- memory: 0.7842240333557129
#--- memory: 0.8716773986816406


In [10]:
# prm = RLHFFlow(model_path=prm_tokenizer_dir, device_map='cuda:0')

# gc.collect();torch.cuda.empty_cache();
# print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
# print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))
# print('#--- memory:', torch.cuda.memory_allocated(2)/(1024**3))
# print('#--- memory:', torch.cuda.memory_allocated(3)/(1024**3))

In [11]:
# del(prm)
gc.collect();torch.cuda.empty_cache();
print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))

#--- memory: 0.7842240333557129
#--- memory: 0.8716773986816406


In [12]:
#  load data 
data_by_levels = load_data_prm800k(data_dir)


# ds_completions = load_completions(completions_dir)

# load random_seeds     
# random_seeds = np.loadtxt("random_seeds.txt").astype("int64")
# random_seeds = [int(seed) for seed in random_seeds]

1: 43
2: 90
3: 105
4: 128
5: 134


In [13]:
# general params
config = Config()
config.n = 16
config.agg_strategy = 'last'

level = '1'
num_questions = len(data_by_levels[level])
num_questions = 20
num_trials = 1
print(f"num_questions = {num_questions}")

# get batch of questions
batch_of_questions = [data_by_levels[level][q_idx]['problem'] for q_idx in range(num_questions)]

# load completions 
completions_dir = f"results/generate_bon_prm800k_level{level}_n16_v11.jsonl"
scores_dir = f"../results/scores_bon_prm800k_level{level}_n16_v11.jsonl"

# compute results
fout = open(scores_dir, 'w', encoding = 'utf-8')
start_time = time.time()
with open(scores_dir, 'w', encoding = 'utf-8') as fout:
    with open(completions_dir, 'r', encoding = 'utf-8') as fin:
        trial_idx = 0
        for line in fin:
            if trial_idx >= num_trials:
                break
                
            trial_data = json.loads(line)
    
            # Compute the scores of completions
            # print(len(trial_data["completions"]))
            # print(len(batch_of_questions))
            # print(trial_data["completions"][0][0])
            scores = prm.score(batch_of_questions, trial_data["completions"][:num_questions])
            agg_scores = [
                [aggregate_scores(s, config.agg_strategy) for s in score] for score in scores
            ]
            # print(agg_scores)
            
            trial_data["agg_scores"] = agg_scores
            # print(trial_data.keys())
            json.dump(trial_data, fout)
            fout.write('\n')
    
            # compute the time
            if trial_idx % 1 == 0:
                total_time = time.time() - start_time
                time_per_trial = total_time/(trial_idx+1)
                time_per_question = time_per_trial/num_questions
                print(f"trial {trial_idx}")
                print(f"it takes {time_per_question:0.4f}s per question")
                print(f"it takes {time_per_trial:0.4f}s per trial")
    
            trial_idx += 1

total_time = time.time() - start_time
print(f"it takes {total_time:0.4f}s in total")

num_questions = 20
[997, 308, 392, 826, 719, 400, 411, 411, 418, 498, 459, 382, 347, 450, 547, 348]
#--- memory: 0.7842240333557129
#--- memory: 0.8716773986816406
#--- memory: 0.8716773986816406
#--- memory: 0.34867429733276367
cuda:0
[175, 210, 287, 200, 190, 235, 219, 310, 223, 301, 344, 391, 284, 187, 197, 289]
#--- memory: 6.695084571838379
#--- memory: 1.0324516296386719
#--- memory: 1.0324516296386719
#--- memory: 0.41746091842651367
cuda:0
[722, 489, 1495, 709, 517, 565, 788, 631, 565, 602, 2469, 748, 583, 555, 429, 518]
#--- memory: 3.1055707931518555
#--- memory: 0.9419517517089844
#--- memory: 0.9419517517089844
#--- memory: 0.38047361373901367
cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.36 GiB. GPU 0 has a total capacity of 31.73 GiB of which 7.79 GiB is free. Including non-PyTorch memory, this process has 23.94 GiB memory in use. Of the allocated memory 17.71 GiB is allocated by PyTorch, and 5.86 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
# gc.collect();torch.cuda.empty_cache();
print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(2)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(3)/(1024**3))

#--- memory: 17.7143816947937
#--- memory: 1.3186912536621094
#--- memory: 1.3186912536621094
#--- memory: 0.5311694145202637
